# Import packages

In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.insert(1, 'C:\\Users\\pyann\\Dropbox (CEDIA)\\CPR\\Model')


pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Import dataset

In [2]:
from CPR import analysis

inputs = analysis.get_dataset()

# make dataset smaller for analysis
inputs = inputs.sample(500)

inputs.head()

,byear,sex,ret_age,education,init_wage,pension,bal_rrsp,bal_tfsa,bal_other_reg,bal_unreg,cont_rate_rrsp,cont_rate_tfsa,cont_rate_other_reg,cont_rate_unreg,withdrawal_rrsp,withdrawal_tfsa,withdrawal_other_reg,withdrawal_unreg,replacement_rate_db,rate_employee_db,income_previous_db,init_dc,rate_employee_dc,rate_employer_dc,s_byear,s_sex,s_ret_age,s_education,s_init_wage,s_pension,s_bal_rrsp,s_bal_tfsa,s_bal_other_reg,s_bal_unreg,s_cont_rate_rrsp,s_cont_rate_tfsa,s_cont_rate_other_reg,s_cont_rate_unreg,s_withdrawal_rrsp,s_withdrawal_tfsa,s_withdrawal_other_reg,s_withdrawal_unreg,s_replacement_rate_db,s_rate_employee_db,s_income_previous_db,s_init_dc,s_rate_employee_dc,s_rate_employer_dc,weight,couple,prov,first_residence,second_residence,price_first_residence,price_second_residence,business,price_business,mix_bonds,mix_bills,mix_equity,fee,fee_equity,credit_card,personal_loan,student_loan,car_loan,credit_line,first_mortgage,second_mortgage,other_debt,credit_card_payment,personal_loan_payment,student_loan_payment,car_loan_payment,credit_line_payment,first_mortgage_payment,second_mortgage_payment,other_debt_payment,claim_age_cpp,s_claim_age_cpp,cap_gains_unreg,s_cap_gains_unreg,realized_losses_unreg,s_realized_losses_unreg,init_room_rrsp,init_room_tfsa,s_init_room_rrsp,s_init_room_tfsa
2397,1986.0,female,55.0,post-secondary,80000.0,NaN,30000.000000,15000.000000,NaN,NaN,0.00,0.000000,NaN,NaN,0.0,0.0,NaN,NaN,0.6,0.04,NaN,NaN,NaN,NaN,1983.0,male,70.0,university,80000.0,NaN,12500.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,True,qc,375000.0,NaN,375000.0,NaN,NaN,NaN,0.000000,1.000000,0.000000,0.006667,0.000000,2000.0,NaN,NaN,20000.0,NaN,184000.0,NaN,NaN,100.0,NaN,NaN,400.0,NaN,1733.0,NaN,NaN,61.0,65.0,0,0.0,0,0.0,0,0,0.0,0.0
376,1984.0,male,55.0,high school,90000.0,NaN,NaN,NaN,NaN,5000.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.0,NaN,NaN,0.0,40000.0,0.04,0.02,1972.0,female,60.0,university,50000.0,NaN,50000.000000,5000.0,NaN,NaN,0.246268,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.7,0.05,NaN,NaN,NaN,NaN,1,True,on,60000.0,250.0,60000.0,250.0,NaN,NaN,0.000000,1.000000,0.000000,0.010000,0.000000,NaN,NaN,8000.0,NaN,12000.0,NaN,NaN,NaN,NaN,NaN,300.0,NaN,1000.0,NaN,NaN,NaN,60.0,70.0,0,0.0,0,0.0,0,0,0.0,0.0
2241,1979.0,male,63.0,post-secondary,40000.0,NaN,114000.000000,100.000000,NaN,5000.000000,0.00,0.000000,NaN,0.000000,0.0,1000.0,NaN,0.0,0.6,0.05,NaN,NaN,NaN,NaN,1979.0,female,60.0,high school,70000.0,NaN,97222.222222,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,True,on,NaN,NaN,NaN,NaN,300.0,300.0,0.375000,0.062500,0.562500,0.014687,0.008437,NaN,1000.0,NaN,21000.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,500.0,NaN,NaN,NaN,NaN,70.0,65.0,0,0.0,0,0.0,0,0,0.0,0.0
3493,1987.0,male,65.0,post-secondary,190000.0,NaN,114285.714286,NaN,NaN,NaN,0.05,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.7,0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,False,qc,90000.0,NaN,90000.0,NaN,NaN,NaN,0.329897,0.113402,0.556701,0.014021,0.008969,NaN,NaN,NaN,14000.0,220000.0,NaN,NaN,NaN,NaN,NaN,NaN,400.0,700.0,NaN,NaN,NaN,70.0,NaN,0,NaN,0,NaN,0,0,NaN,NaN
2466,1971.0,male,68.0,university,52000.0,NaN,0.000000,3571.428571,NaN,6690.140845,0.00,0.017007,NaN,0.003521,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,False,on,240000.0,400000.0,240000.0,400000.0,NaN,NaN,0.246914,0.012346,0.740741,0.018593,0.014815,NaN,NaN,NaN,NaN,NaN,65000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750.0,NaN,NaN,65.0,NaN,0,NaN,0,NaN,0,0,NaN,NaN


# Run model
Deterministic model and stochastic model with 25 simulations

In [ ]:
from CPR import main

res_deter = main.run_simulations(inputs, 1, non_stochastic=True,
                                 multiprocessing=True)
res_deter.summarize()


res_stoch = main.run_simulations(inputs, 20, non_stochastic=False,
                                 multiprocessing=True)
res_stoch.summarize()

# Analyse results

## See output

In [ ]:
res_deter.output.head()

## Merge with input and check preparedness

In [ ]:
res_deter.merge()
res_deter.check_preparedness(factor_couple=np.sqrt(2), cons_floor=5000)
df = res_deter.df_merged

print(f'percentage people prepared in sample: {df.prepared.mean() * 100}%')

## RRI distribution

In [ ]:
sns.set()
ax = df.rri[df.rri < 500].hist()
ax.set_title('RRI distribution')
plt.show()

## Risk
### Preparation rate at aggregate level

In [ ]:
res_stoch.merge()
res_stoch.check_preparedness(factor_couple=np.sqrt(2), cons_floor=5000)
df = res_stoch.df_merged

In [ ]:
ax = df.groupby('sim')['prepared'].mean().hist()
ax.grid()
ax.set_title('rri distribution')
ax.set_xlabel('percentage people prepared')
ax.grid()
plt.show()

## Experiments
### Changes in returns

In [ ]:
vars(res_deter.prices)

In [ ]:
factors = np.linspace(0, 2, 10)

l_prepared = []
for factor in factors:
    res = main.run_simulations(inputs, 1, non_stochastic=True,
                                     multiprocessing=True,
                                     mu_equity=factor * 0.0688,
                                     mu_bills=factor * 0.0103,
                                     mu_bonds=factor * 0.0253)
    res.check_preparedness(factor_couple=np.sqrt(2), cons_floor=5000)
    l_prepared.append(res.df_merged.prepared.mean())    

In [ ]:
plt.plot(factors, l_prepared)
plt.title('changes in returns')
plt.xlabel('factor')
plt.ylabel('percentage prepared')
plt.show()

### Selling first residence

In [ ]:
vars(res_deter.common)

In [ ]:
res_not_selling = main.run_simulations(inputs, 1, non_stochastic=True,
                                       multiprocessing=True,
                                       sell_first_resid=False)
res_not_selling.check_preparedness(factor_couple=np.sqrt(2), cons_floor=5000)
df = res_not_selling.df_merged

print(f'percentage people prepared without selling house: {df.prepared.mean() * 100}%')

res_selling = main.run_simulations(inputs, 1, non_stochastic=True,
                                   multiprocessing=True,
                                   sell_first_resid=True)
res_selling.check_preparedness(factor_couple=np.sqrt(2), cons_floor=5000)
df = res_selling.df_merged

print(f'percentage people prepared when selling house: {df.prepared.mean() * 100}%')

# Using CPR with other inputs

add link to documentation 
More info on the variables here: http://ire.hec.ca/en/wp-content/uploads/sites/3/2020/06/cpr-report-2020-final.pdf

In [ ]:
inputs.head()

In [ ]:
# saving copy of dataframe
inputs.iloc[0, :].to_frame().T.to_csv('my_inputs.csv', index=False)

In [ ]:
my_inputs = pd.read_csv('my_inputs.csv')

In [ ]:
my_res = main.run_simulations(my_inputs, 500, non_stochastic=False,
                              multiprocessing=True)
my_res.check_preparedness()
df = my_res.df_merged

df.prepared.value_counts(normalize=True)

In [ ]:
ax = sns.distplot(df.loc[df.rri<500, 'rri'])
ax.grid()
ax.set_title('rri distribution')
ax.grid()
plt.show()